In [2]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import LongformerTokenizerFast, LongformerForQuestionAnswering, AdamW
from transformers import Trainer, TrainingArguments

In [ ]:
from SQUAD4L import obtain_dataset
from datetime import datetime
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# SETUP
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
path1 = 'data/squad/train-v2.0.json'
path2 = 'data/squad/dev-v2.0.json'
tokenizer = LongformerTokenizerFast.from_pretrained('allenai/longformer-base-4096')
model = LongformerForQuestionAnswering.from_pretrained('allenai/longformer-base-4096')
writer = SummaryWriter()
model_save_path = f'models/{datetime.now().strftime("%b-%d-%Y-%H%M%S")}'
batch_size = 2
verbose = True

In [ ]:
train_dataset, val_dataset = obtain_dataset(path1, path2, num_samples_train=80, num_samples_val=20)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
optim = AdamW(model.parameters(), lr=5e-5)

In [ ]:
for i, batch in enumerate(train_loader):
    optim.zero_grad()
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    start_positions = batch['start_positions'].to(device)
    end_positions = batch['end_positions'].to(device)
    outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions, output_attentions=True)
    loss = outputs.loss
    writer.add_scalar("Loss/train", loss, epoch)
    loss.backward()
    optim.step()
    print(f'Step {i} - loss: {loss:.3}')

In [ ]:
%tensorboard --logdir=runs